In [1]:
# !pip install keras_cv_attention_models

In [4]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets, GhostFaceNets_with_Bias
import tensorflow as tf
import keras_cv_attention_models

2024-05-10 11:01:56.673622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 11:01:56.721512: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 11:01:56.721833: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


2024-05-10 11:02:00.165462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


GhostFaceNetV1

In [16]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/bk_logs_celeb_112x112'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [17]:
#GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation_96 --> activation_96
>>>> Convert ReLU: activation_97 --> activation_97
>>>> Convert ReLU: activation_98 --> activation_98
>>>> Convert ReLU: activation_99 --> activation_99
>>>> Convert ReLU: activation_100 --> activation_100
>>>> Convert ReLU: activation_101 --> activation_101
>>>> Convert ReLU: activation_102 --> activation_102
>>>> Convert ReLU: activation_103 --> activation_103
>>>> Convert ReLU: activation_104 --> activation_104
>>>> Convert ReLU: activation_105 --> activation_105
>>>> Convert ReLU: activation_107 --> activation_107
>>>> Convert ReLU: activation_108 --> activation_108
>>>> Convert ReLU: activation_109 --> activation_109
>>>> Convert ReLU: activation_111 --> activation_111
>>>> Convert ReLU: activation_112 --> activation_112
>>>> Convert ReLU: activation_113 --> activation_113
>>>> Convert ReLU: activation_114 --> activation_114
>>>> Convert ReLU: activation_115 --> ac

In [22]:
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=32, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
>>>> L2 regularizer value from basic_model: 0.00025


In [23]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 1},
]
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: bk_logs_celeb_112x112_shuffle.npz
>>>> Image length: 2818, Image class length: 2818, classes: 85
>>>> Use specified optimizer: <keras.src.optimizers.sgd.SGD object at 0x770b193e7f40>
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': -88, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}

Learning rate for iter 1 is 0.10000000149011612, global_iterNum is 0


2024-05-10 11:14:22.317744: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 48.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-10 11:14:22.345566: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 48.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-10 11:14:23.910777: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 75.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-10 11:14:23.910837: W external/local_tsl/tsl/framework/b

23/88 [======>.......................] - ETA: 2:24 - loss: 23.6760 - accuracy: 0.6399

2024-05-10 11:15:58.193678: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 49.00MiB (rounded to 51380224)requested by op gradient_tape/model_2/depthwise_conv2d_126/depthwise/DepthwiseConv2dNativeBackpropInput
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-05-10 11:15:58.194049: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-05-10 11:15:58.194086: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 1046, Chunks in use: 1046. 261.5KiB allocated for chunks. 261.5KiB in use in bin. 63.8KiB client-requested in use in bin.
2024-05-10 11:15:58.194110: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 441, Chunks in use: 440. 255.5KiB allocated for chun

ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/model_2/depthwise_conv2d_126/depthwise/DepthwiseConv2dNativeBackpropInput defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ky/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ky/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ky/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ky/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/ky/.local/lib/python3

GhostFaceNetV2

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
#GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

In [ ]:
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

CosFaceLoss

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.CosFaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.CosFaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

Subcenter ArcFace Loss

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

In [12]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
#GhostFaceNetV1
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [13]:
#GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025


ValueError: When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5` (Keras weights format). Received: filepath=checkpoints/ghostnetv2_w1.3_s2_topk.h5

In [ ]:
""" First, Train with `lossTopK = 3` """
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer, "lossTopK": 3},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "lossTopK": 3},
]
tt.train(sch, 0)

In [ ]:
""" Then drop non-dominant subcenters and high-confident noisy data, which is `>75 degrees` """
import data_drop_top_k
# data_drop_top_k.data_drop_top_k('./checkpoints/TT_mobilenet_topk_bs256.h5', '/datasets/faces_casia_112x112_folders/', limit=20)
new_data_path = data_drop_top_k.data_drop_top_k(tt.model, tt.data_path)

In [ ]:
""" Train with the new dataset again, this time `lossTopK = 1` """
tt.reset_dataset(new_data_path)

At this point you can continue training the model or start training again and considering the generated dataset as a new dataset

In [ ]:
# Continue training
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [ ]:
# Train the models from scatch using the new generated dataset

# New Cleaned Dataset
data_path = 'Path_To_New_Generated_Dataset'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV1
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
# GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)